In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import numpy as np

In [2]:
os.getcwd()

'/Users/regan/Documents/Springboard/Capstone2.0/Final Versions'

In [5]:
#load 2019 play-by-play dataset
pbp2019 = pd.read_csv('../2019_CFB_Data.csv', encoding='latin-1')
pbp2019.head()

,gameid,provider,o/u,hometeam,home money,home spread,awayteam,away money,away spread,playid,...,fumble,td,xp result,two pt result,playstring,play str len,complex,max len,max spaces,week
0,401112431,teamrankings,55.0,Florida State Seminoles,-234.0,-6.0,Boise State Broncos,200.0,6.0,102925701,...,True,False,NaN,NaN,James Blackman sacked by Kekaula Kaniho and TE...,223,5,33,5,1
1,401110725,teamrankings,51.0,LSU Tigers,-3019.0,-27.0,Georgia Southern Eagles,1487.0,27.0,101998701,...,True,False,NaN,NaN,Shai Werts sacked by Rashard Lawrence for a lo...,212,5,33,5,1
2,401110723,teamrankings,46.0,Florida Gators,-291.0,-7.0,Miami Hurricanes,244.0,7.0,104966901,...,True,False,NaN,NaN,Jarren Williams sacked by Trey Dean III for a ...,192,5,33,5,1
3,401121935,consensus,49.0,Troy Trojans,NaN,-35.0,Campbell Fighting Camels,NaN,35.0,103979801,...,True,False,NaN,NaN,Team run for a loss of 26 yards for a SAFETY T...,101,5,33,5,1
4,401114228,teamrankings,65.5,Washington State Cougars,-5595.0,-33.0,New Mexico State Aggies,2219.0,33.0,101937404,...,False,False,NaN,NaN,Anthony Gordon pass complete to Brandon Arcona...,63,2,33,5,1


# Drop Columns Here

In [6]:
#drop unnecessary columns
drop_cols = ['o/u', 'home money', 'home spread', 'away money', 'away spread', 'playstring', 'play str len', 'max len', 'max spaces', 'provider','activeid','playyds','complex','playtype']
pbp2019_red = pbp2019.drop(drop_cols, 1)

In [7]:
#convert team names to lowercase
pbp2019_red['hometeam'] = pbp2019_red['hometeam'].str.lower()
pbp2019_red['awayteam'] = pbp2019_red['awayteam'].str.lower()
pbp2019_red['activeteam'] = pbp2019_red['activeteam'].str.lower()
pbp2019_red['nonactive team'] = pbp2019_red['nonactive team'].str.lower()

pbp2019_red.head()

,gameid,hometeam,awayteam,playid,activeteam,nonactive team,drive #,quarter,sec,homescore,...,placekicker,pkyds,pkresult,penalty,int,fumble,td,xp result,two pt result,week
0,401112431,florida state seminoles,boise state broncos,102925701,florida state seminoles,boise state broncos,5,2,462,24,...,NaN,NaN,NaN,False,False,True,False,NaN,NaN,1
1,401110725,lsu tigers,georgia southern eagles,101998701,georgia southern eagles,lsu tigers,3,1,12,21,...,NaN,NaN,NaN,False,False,True,False,NaN,NaN,1
2,401110723,florida gators,miami hurricanes,104966901,miami hurricanes,florida gators,11,4,210,24,...,NaN,NaN,NaN,False,False,True,False,NaN,NaN,1
3,401121935,troy trojans,campbell fighting camels,103979801,campbell fighting camels,troy trojans,9,3,121,36,...,NaN,NaN,NaN,False,False,True,False,NaN,NaN,1
4,401114228,washington state cougars,new mexico state aggies,101937404,washington state cougars,new mexico state aggies,2,1,385,7,...,NaN,NaN,NaN,False,False,False,False,NaN,NaN,1


In [8]:
#load conference membership data
conferences = pd.read_csv('../Other_Data/Conferences.csv',skip_blank_lines=True).fillna(np.nan) 

#conver to lowercase
conferences = conferences.apply(lambda x: x.astype(str).str.lower())

conferences.head()

,ACC,Big Ten,Big 12,Pac-12,SEC
0,boston college,illinois,baylor,arizona,alabama
1,clemson,indiana,iowa state,arizona state,arkansas
2,duke,iowa,kansas,california,auburn
3,florida state,maryland,kansas state,ucla,florida
4,georgia tech,michigan,oklahoma,colorado,georgia


The conference dataframe does not include the mascot name with the team name while the play-by-play dataset does. 

In [9]:
# load name and mascot file

name_mascot = pd.read_csv('../Other_Data/Names_Mascots.txt')
name_mascot.head()

,name,mascot
0,boston college,boston college eagles
1,clemson,clemson tigers
2,duke,duke blue devils
3,florida state,florida state seminoles
4,georgia tech,georgia tech yellow jackets


In [11]:
#convert to dict
mascot_dict = name_mascot.set_index('name')['mascot'].to_dict()

#adding nan to dictionary
mascot_dict['nan']=np.nan

In [12]:
#changing team name in conferences to include mascot using the mascot dictionary

for conf in conferences:
    conferences[conf] = conferences[conf].apply(lambda x: mascot_dict[x])

In [18]:
#convert conferences to dictionary
conf_dict = conferences.to_dict(orient='list')

#reverse dconferences dictionary
conf_rev = {v: k for k, values in conf_dict.items() for v in values}

#convert to lowercase
keys_values = conf_rev.items()
conf_rev = {str(key).lower(): str(value).lower() for key, value in keys_values}

In [20]:
#add home conference column to play-by-play dataset
pbp2019_red['home_conf']=np.nan
for k, v in conf_rev.items():
    for i in range(len(pbp2019_red['hometeam'])):
        if k in pbp2019_red['hometeam'][i]:
            pbp2019_red['home_conf'][i] = v

/Users/regan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/regan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [21]:
#add away conference column
pbp2019_red['away_conf']=np.nan
for k, v in conf_rev.items():
    for i in range(len(pbp2019_red['awayteam'])):
        if k in pbp2019_red['awayteam'][i]:
            pbp2019_red['away_conf'][i] = v

/Users/regan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
#add active team conference column
pbp2019_red['active_conf']=np.nan
for k, v in conf_rev.items():
    for i in range(len(pbp2019_red['activeteam'])):
        if k in pbp2019_red['activeteam'][i]:
            pbp2019_red['active_conf'][i]=v

/Users/regan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
pbp2019_red.head()

,gameid,hometeam,awayteam,playid,activeteam,nonactive team,drive #,quarter,sec,homescore,...,penalty,int,fumble,td,xp result,two pt result,week,home_conf,away_conf,active_conf
0,401112431,florida state seminoles,boise state broncos,102925701,florida state seminoles,boise state broncos,5,2,462,24,...,False,False,True,False,NaN,NaN,1,acc,NaN,acc
1,401110725,lsu tigers,georgia southern eagles,101998701,georgia southern eagles,lsu tigers,3,1,12,21,...,False,False,True,False,NaN,NaN,1,sec,NaN,NaN
2,401110723,florida gators,miami hurricanes,104966901,miami hurricanes,florida gators,11,4,210,24,...,False,False,True,False,NaN,NaN,1,sec,acc,acc
3,401121935,troy trojans,campbell fighting camels,103979801,campbell fighting camels,troy trojans,9,3,121,36,...,False,False,True,False,NaN,NaN,1,NaN,NaN,NaN
4,401114228,washington state cougars,new mexico state aggies,101937404,washington state cougars,new mexico state aggies,2,1,385,7,...,False,False,False,False,NaN,NaN,1,pac-12,NaN,pac-12


In [24]:
#drop all games that do not involve a team on offense in the Big Ten or SEC
c = ['big ten','sec']
pbp = pbp2019_red.loc[pbp2019_red['active_conf'].isin(c)]

In [29]:
#load rankings dataset
rankings2019 = pd.read_csv('../Other_Data/Ranking.csv')
rankings2019.head(10)

,PreseasonAug 1[18],Week 1Sep 3[19],Week 2Sep 8[20],Week 3Sep 15[21],Week 4Sep 22[22],Week 5Sep 29[23],Week 6Oct 6[24],Week 7Oct 13[25],Week 8Oct 20[26],Week 9Oct 27[27],Week 10Nov 3[28],Week 11Nov 10[29],Week 12Nov 17[30],Week 13Nov 24[31],Week 14Dec 1[32],Week 15Dec 8[33],Week 16 (Final)Jan 14[34],Unnamed: 17
0,Clemson (59),Clemson (1–0) (58),Clemson (2–0) (60),Clemson (3–0) (62),Clemson (4–0) (62),Alabama (5–0) (29),Alabama (5–0) (42),Alabama (6–0) (44),Alabama (7–0) (44),Alabama (8–0) (40),Alabama (8–0) (37),LSU (9–0) (55),LSU (10–0) (55),LSU (11–0) (52),LSU (12–0) (43),LSU (13–0) (46),LSU (15–0) (65),1
1,Alabama (6),Alabama (1–0) (6),Alabama (2–0) (3),Alabama (3–0) (3),Alabama (4–0) (2),Clemson (5–0) (30),Clemson (5–0) (20),Clemson (6–0) (14),Clemson (7–0) (10),LSU (8–0) (7),LSU (8–0) (11),Ohio State (9–0) (5),Ohio State (10–0) (6),Ohio State (11–0) (7),Ohio State (12–0) (17),Ohio State (13–0) (14),Clemson (14–1),2
2,Georgia,Georgia (1–0),Georgia (2–0),Georgia (3–0),Georgia (4–0) (1),Georgia (4–0) (1),Georgia (5–0),LSU (6–0) (3),LSU (7–0) (3),Clemson (8–0) (10),Clemson (9–0) (9),Clemson (10–0) (4),Clemson (11–0) (4),Clemson (11–0) (4),Clemson (12–0) (4),Clemson (13–0) (5),Ohio State (13–1),3
3,Oklahoma,Oklahoma (1–0),Oklahoma (2–0),Oklahoma (3–0),Oklahoma (3–0),Oklahoma (4–0),Ohio State (5–0) (3),Ohio State (6–0) (4),Ohio State (7–0) (8),Ohio State (8–0) (8),Ohio State (8–0) (8),Alabama (8–1),Georgia (9–1),Georgia (10–1),Georgia (11–1),Oklahoma (12–1),Georgia (12–2),4
4,Ohio State,Ohio State (1–0),LSU (2–0),LSU (3–0),LSU (4–0),Ohio State (5–0) (4),Oklahoma (5–0),Oklahoma (6–0),Oklahoma (7–0),Penn State (8–0),Penn State (8–0),Georgia (8–1),Alabama (9–1),Alabama (10–1),Utah (11–1),Georgia (11–2),Oregon (12–2),5
5,LSU,LSU (1–0),Ohio State (2–0),Ohio State (3–0),Ohio State (4–0),LSU (4–0),LSU (5–0),Wisconsin (6–0),Penn State (7–0),Florida (7–1),Georgia (7–1),Oregon (8–1),Oregon (9–1),Utah (10–1),Oklahoma (11–1),Oregon (11–2),Oklahoma (12–2),6
6,Michigan,Michigan (1–0),Notre Dame (2–0),Notre Dame (2–0),Auburn (4–0),Auburn (4–0),Florida (6–0),Penn State (6–0),Notre Dame (5–1),Georgia (6–1),Oregon (8–1),Minnesota (9–0),Oklahoma (9–1),Oklahoma (10–1),Florida (10–2),Florida (10–2),Florida (11–2),7
7,Florida,Notre Dame (1–0),Florida (2–0),Florida (3–0),Florida (4–0),Florida (5–0),Wisconsin (5–0),Notre Dame (5–1),Florida (7–1),Oregon (7–1),Oklahoma (7–1),Oklahoma (8–1),Utah (9–1),Florida (9–2),Baylor (11–1),Baylor (11–2),Alabama (11–2),8
8,Notre Dame,Texas (1–0),Auburn (2–0),Auburn (3–0),Wisconsin (3–0),Wisconsin (4–0),Penn State (5–0),Florida (6–1),Georgia (6–1),Oklahoma (7–1),Utah (8–1),Utah (8–1),Penn State (9–1),Minnesota (10–1),Alabama (10–2),Alabama (10–2),Penn State (11–2),9
9,Texas,Florida (1–0),Michigan (2–0),Michigan (2–0),Notre Dame (2–1),Notre Dame (3–1),Notre Dame (4–1),Georgia (5–1),Auburn (6–1),Utah (7–1),Baylor (8–0),Baylor (9–0),Florida (9–2),Baylor (10–1),Wisconsin (10–2),Utah (11–2),Minnesota (11–2),10


In [42]:
## cleaning the rankings dataframe

#save column names
rankings2019col = rankings2019.columns

#remove []
rankings2019col_rename = [re.sub('\[.*?\]', '', name) for name in rankings2019col]

#remove date
rankings2019col_rename2 = [re.sub('Aug [0-9]*| Jan [0-9]*|Sep [0-9]*|Oct [0-9]*|Nov [0-9]*|Dec [0-9]*|Jan [0-9]*','',month) for month in rankings2019col_rename]

#remove "week"
rankings2019col_3 = [re.sub('Week','',week)for week in rankings2019col_rename2]

#replace columns with new names
rankings2019.columns = rankings2019col_3

#remove (record) and (votes) from data
rankings = rankings2019.replace(to_replace = '\(.*?\)', value = '', regex=True)

#drop preseason and postseason rankings
drop_col_rank = ['Preseason',' 15',' 16 (Final)','Unnamed: 17']
rank = rankings.drop(drop_col_rank, 1)

#convert column names to integers
colnames = [1,2,3,4,5,6,7,8,9,10,11,12,13, 14]
col = rank.columns
rank = rank.rename(columns = dict(zip(col, colnames)))

#convert to lowercase 
rank = rank.apply(lambda x: x.astype(str).str.lower())

#rename schools in rank datset to include mascot name
for school, mascot in mascot_dict.items():
    for team in rank:
        for i in range(len(rank)):
            if school in rank[team][i]:
                rank[team][i] = mascot

rank.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,clemson tigers,clemson tigers,clemson tigers,clemson tigers,alabama crimson tide,alabama crimson tide,alabama crimson tide,alabama crimson tide,alabama crimson tide,alabama crimson tide,lsu tigers,lsu tigers,lsu tigers,lsu tigers
1,alabama crimson tide,alabama crimson tide,alabama crimson tide,alabama crimson tide,clemson tigers,clemson tigers,clemson tigers,clemson tigers,lsu tigers,lsu tigers,ohio state buckeyes,ohio state buckeyes,ohio state buckeyes,ohio state buckeyes
2,georgia bulldogs,georgia bulldogs,georgia bulldogs,georgia bulldogs,georgia bulldogs,georgia bulldogs,lsu tigers,lsu tigers,clemson tigers,clemson tigers,clemson tigers,clemson tigers,clemson tigers,clemson tigers
3,oklahoma sooners,oklahoma sooners,oklahoma sooners,oklahoma sooners,oklahoma sooners,ohio state buckeyes,ohio state buckeyes,ohio state buckeyes,ohio state buckeyes,ohio state buckeyes,alabama crimson tide,georgia bulldogs,georgia bulldogs,georgia bulldogs
4,ohio state buckeyes,lsu tigers,lsu tigers,lsu tigers,ohio state buckeyes,oklahoma sooners,oklahoma sooners,oklahoma sooners,penn state nittany lions,penn state nittany lions,georgia bulldogs,alabama crimson tide,alabama crimson tide,utah utes


In [43]:
#create ranking dictionary

rank_25 = rank

#reset index to correspond to ranking
rank_25.index = np.arange(1, len(rank_25) +1)

#convert rank_25 to dict
rank_dict = rank_25.to_dict()

#reverse inner dictionary
from collections import defaultdict
import pprint

flipped = defaultdict(dict)
for week, spots in rank_dict.items():
    for spot, team in spots.items():
        flipped[week][team]= spot

pprint.pprint(dict(flipped))

{1: {'alabama crimson tide': 2,
     'auburn tigers': 13,
     'boise state ': 24,
     'clemson tigers': 1,
     'florida gators': 10,
     'georgia bulldogs': 3,
     'iowa hawkeyes': 19,
     'lsu tigers': 6,
     'michigan wolverines': 20,
     'nebraska cornhuskers': 25,
     'notre dame ': 8,
     'ohio state buckeyes': 5,
     'oklahoma sooners': 4,
     'oregon ducks': 18,
     'penn state nittany lions': 14,
     'stanford cardinal': 23,
     'syracuse orange': 22,
     'texas longhorns': 11,
     'ucf ': 17,
     'utah utes': 15,
     'washington huskies': 21,
     'wisconsin badgers': 16},
 2: {'alabama crimson tide': 2,
     'auburn tigers': 9,
     'boise state ': 22,
     'clemson tigers': 1,
     'florida gators': 8,
     'georgia bulldogs': 3,
     'iowa hawkeyes': 18,
     'lsu tigers': 5,
     'maryland terrapins': 25,
     'michigan wolverines': 19,
     'mississippi state bulldogs': 23,
     'notre dame ': 7,
     'ohio state buckeyes': 6,
     'oklahoma sooners': 4

In [47]:
#add home rank and away rank columns to play-by-play dataset 

replaced_dfs = []
for week, pbp_df in pbp.groupby('week'):
    pbp_df['home_rank'] = pbp_df['hometeam'].map(flipped[week])
    replaced_dfs.append(pbp_df)
    
pbp_rank = pd.concat(replaced_dfs)

#add away rank column
replaced_away_dfs = []
for week, pbp_df in pbp_rank.groupby('week'):
    pbp_df['away_rank'] = pbp_df['awayteam'].map(flipped[week])
    replaced_away_dfs.append(pbp_df)
    
pbp_rank = pd.concat(replaced_away_dfs)
pbp_rank.head(25)

,gameid,hometeam,awayteam,playid,activeteam,nonactive team,drive #,quarter,sec,homescore,...,fumble,td,xp result,two pt result,week,home_conf,away_conf,active_conf,home_rank,away_rank
86,401110730,tennessee volunteers,georgia state panthers,104857801,tennessee volunteers,georgia state panthers,8,4,861,20,...,False,False,NaN,NaN,1,sec,NaN,sec,NaN,NaN
87,401110730,tennessee volunteers,georgia state panthers,104994406,tennessee volunteers,georgia state panthers,12,4,55,23,...,False,False,NaN,NaN,1,sec,NaN,sec,NaN,NaN
88,401110730,tennessee volunteers,georgia state panthers,101884406,tennessee volunteers,georgia state panthers,2,1,715,0,...,False,False,NaN,NaN,1,sec,NaN,sec,NaN,NaN
89,401110730,tennessee volunteers,georgia state panthers,101979201,tennessee volunteers,georgia state panthers,3,1,127,7,...,False,False,NaN,NaN,1,sec,NaN,sec,NaN,NaN
90,401110732,vanderbilt commodores,georgia bulldogs,102876901,vanderbilt commodores,georgia bulldogs,2,2,750,0,...,False,False,NaN,NaN,1,sec,sec,sec,NaN,3.0
94,401110732,vanderbilt commodores,georgia bulldogs,102866901,vanderbilt commodores,georgia bulldogs,2,2,810,0,...,False,False,NaN,NaN,1,sec,sec,sec,NaN,3.0
107,401110724,kentucky wildcats,toledo rockets,104966902,kentucky wildcats,toledo rockets,14,4,210,38,...,True,False,NaN,NaN,1,sec,NaN,sec,NaN,NaN
109,401110724,kentucky wildcats,toledo rockets,102887404,kentucky wildcats,toledo rockets,3,2,685,7,...,False,False,NaN,NaN,1,sec,NaN,sec,NaN,NaN
111,401110724,kentucky wildcats,toledo rockets,101878403,kentucky wildcats,toledo rockets,1,1,735,0,...,False,False,NaN,NaN,1,sec,NaN,sec,NaN,NaN
113,401110724,kentucky wildcats,toledo rockets,103854604,kentucky wildcats,toledo rockets,6,3,893,14,...,False,False,NaN,NaN,1,sec,NaN,sec,NaN,NaN


In [54]:
#convert boolean columns to int
pbp_rank['penalty'] = pbp_rank['penalty'].astype(int)
pbp_rank['int']=pbp_rank['int'].astype(int)
pbp_rank['td']=pbp_rank['td'].astype(int)
pbp_rank['fumble']=pbp_rank['fumble'].astype(int)

In [55]:
#show all columns
from IPython.display import display

pd.set_option('display.max_columns', None)

# ????

In [50]:
for i in range(len(pbp2019_red['hometeam'])):
        if k in pbp2019_red['hometeam'][i]:
            pbp2019_red['home_conf'][i] = v

/Users/regan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
# create yards gained column
pbp_rank['yards'] = pbp_rank['recyds'].fillna(0) + pbp_rank['runyds'].fillna(0)

In [59]:
# score or not column
# includes both td and pk, but not the extra point
pbp_rank['scored'] = np.nan

for i in range(len(pbp_rank['scored'])):
    if pbp_rank['td'].iloc[i] == 1:
        pbp_rank['scored'].iloc[i] = 1
    elif pbp_rank['pkresult'].iloc[i] == 'GOOD':
        pbp_rank['scored'].iloc[i] = 1
    else:
        pbp_rank['scored'].iloc[i] = 0

/Users/regan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [60]:
# gain / loss / neutral column
pbp_rank['gain_loss'] = np.nan

for i in range(len(pbp_rank['gain_loss'])):
    if pbp_rank['yards'].iloc[i] > 0:
        pbp_rank['gain_loss'].iloc[i] = 'gain'
    elif pbp_rank['yards'].iloc[i] < 0:
        pbp_rank['gain_loss'].iloc[i] = 'loss'
    else:
        pbp_rank['gain_loss'].iloc[i] = 'no yards'

In [62]:
# column for if active team is playing a home game
pbp_rank['is_home']  = np.nan
for i in range(len(pbp_rank['is_home'])):
    if pbp_rank['activeteam'].iloc[i] == pbp_rank['hometeam'].iloc[i]:
        pbp_rank['is_home'].iloc[i] = 1
    else:
        pbp_rank['is_home'].iloc[i] = 0

/Users/regan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [63]:
#create a column with the active (offensive) team's score

pbp_rank['off_team_score'] = np.nan
for i in range(len(pbp_rank['off_team_score'])):
    if pbp_rank['activeteam'].iloc[i] == pbp_rank['hometeam'].iloc[i]:
        pbp_rank['off_team_score'].iloc[i] = pbp_rank['homescore'].iloc[i]
    else:
        pbp_rank['off_team_score'].iloc[i] = pbp_rank['awayscore'].iloc[i]

In [64]:
#create a column with the active (offensive) team's score

pbp_rank['def_team_score'] = np.nan
for i in range(len(pbp_rank['def_team_score'])):
    if pbp_rank['nonactive team'].iloc[i] == pbp_rank['hometeam'].iloc[i]:
        pbp_rank['def_team_score'].iloc[i] = pbp_rank['homescore'].iloc[i]
    else:
        pbp_rank['def_team_score'].iloc[i] = pbp_rank['awayscore'].iloc[i]

In [66]:
#offensive team's rank column

pbp_rank['off_team_rank'] = np.nan
for i in range(len(pbp_rank['off_team_rank'])):
    if pbp_rank['activeteam'].iloc[i] == pbp_rank['hometeam'].iloc[i]:
        pbp_rank['off_team_rank'].iloc[i] = pbp_rank['home_rank'].iloc[i]
    else:
        pbp_rank['off_team_rank'].iloc[i] = pbp_rank['away_rank'].iloc[i]

/Users/regan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [102]:
#defensive team's rank columm

pbp_rank['def_team_rank'] = np.nan
for i in range(len(pbp_rank['def_team_rank'])):
    if pbp_rank['nonactive team'].iloc[i] == pbp_rank['hometeam'].iloc[i]:
        pbp_rank['def_team_rank'].iloc[i] = pbp_rank['home_rank'].iloc[i]
    else:
        pbp_rank['def_team_rank'].iloc[i] = pbp_rank['away_rank'].iloc[i]

In [103]:
#create run/pass column

pbp_rank['run_pass']=np.nan

qbs = pbp_rank['qb'].unique()
runners = pbp_rank['runner'].unique()

for i in range(len(pbp_rank['run_pass'])):
    if pbp_rank['qb'].iloc[i] in qbs:
        pbp_rank['run_pass'].iloc[i] = 'pass'
    if pbp_rank['runner'].iloc[i] in runners:
        pbp_rank['run_pass'].iloc[i] = 'run'

        
pbp_rank['run_pass'].fillna(value='other',inplace=True)

In [105]:
#create rank category columns

pbp_rank['rank_group'] = np.nan
top5 = [1,2,3,4,5]
five_ten = [6,7,8,9,10]
ten_fifteen = [11,12,13,14,15]
fifteen_20 = [16,17,18,19,20]
tw_25 = [21, 22, 23, 24, 25]

#offense

for i in range(len(pbp_rank['rank_group'])):
    if pbp_rank['off_team_rank'].iloc[i] in top5:
        pbp_rank['rank_group'].iloc[i] = 'top 5'
    elif pbp_rank['off_team_rank'].iloc[i] in five_ten:
        pbp_rank['rank_group'].iloc[i] = '6 to 10'
    elif pbp_rank['off_team_rank'].iloc[i] in ten_fifteen:
        pbp_rank['rank_group'].iloc[i] = '11 to 15'
    elif pbp_rank['off_team_rank'].iloc[i] in fifteen_20:
        pbp_rank['rank_group'].iloc[i] = '16 to 20'
    elif pbp_rank['off_team_rank'].iloc[i] in tw_25:
        pbp_rank['rank_group'].iloc[i] = '21 to 25'
    else:
        pbp_rank['rank_group'].iloc[i] = 'unranked'
        


/Users/regan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [106]:
#defense
pbp_rank['rank_group_def'] = np.nan


for i in range(len(pbp_rank['rank_group_def'])):
    if pbp_rank['def_team_rank'].iloc[i] in top5:
        pbp_rank['rank_group_def'].iloc[i] = 'top 5'
    elif pbp_rank['def_team_rank'].iloc[i] in five_ten:
        pbp_rank['rank_group_def'].iloc[i] = '6 to 10'
    elif pbp_rank['def_team_rank'].iloc[i] in ten_fifteen:
        pbp_rank['rank_group_def'].iloc[i] = '11 to 15'
    elif pbp_rank['def_team_rank'].iloc[i] in fifteen_20:
        pbp_rank['rank_group_def'].iloc[i] = '16 to 20'
    elif pbp_rank['def_team_rank'].iloc[i] in tw_25:
        pbp_rank['rank_group_def'].iloc[i] = '21 to 25'
    else:
        pbp_rank['rank_group_def'].iloc[i] = 'unranked'

In [107]:
pbp_rank.head()

,gameid,hometeam,awayteam,playid,activeteam,nonactive team,drive #,quarter,sec,homescore,awayscore,down,distance,yardstoendzone,zone,qb,pass yds,receiver,recyds,runner,runyds,kicker,kickyds,kickreturner,kickretyds,placekicker,pkyds,pkresult,penalty,int,fumble,td,xp result,two pt result,week,home_conf,away_conf,active_conf,home_rank,away_rank,pass,gain_loss,yards,scored,is_home,off_team_score,def_team_score,off_team_rank,def_team_rank,run_pass,rank_group,rank_group_def
86,401110730,tennessee volunteers,georgia state panthers,104857801,tennessee volunteers,georgia state panthers,8,4,861,20,21,1,10,76,GREEN,Jarrett Guarantano,54.0,Dominick Wood-Anderson,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,1,sec,NaN,sec,NaN,NaN,Jarrett Guarantano,gain,54.0,0.0,1.0,20.0,21.0,NaN,NaN,pass,unranked,unranked
87,401110730,tennessee volunteers,georgia state panthers,104994406,tennessee volunteers,georgia state panthers,12,4,55,23,38,3,4,36,ORANGE,Jarrett Guarantano,18.0,Dominick Wood-Anderson,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,1,sec,NaN,sec,NaN,NaN,Jarrett Guarantano,gain,18.0,0.0,1.0,23.0,38.0,NaN,NaN,pass,unranked,unranked
88,401110730,tennessee volunteers,georgia state panthers,101884406,tennessee volunteers,georgia state panthers,2,1,715,0,7,3,4,62,GREEN,Jarrett Guarantano,7.0,Dominick Wood-Anderson,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,1,sec,NaN,sec,NaN,NaN,Jarrett Guarantano,gain,7.0,0.0,1.0,0.0,7.0,NaN,NaN,pass,unranked,unranked
89,401110730,tennessee volunteers,georgia state panthers,101979201,tennessee volunteers,georgia state panthers,3,1,127,7,7,1,10,46,YELLOW,Jarrett Guarantano,0.0,Dominick Wood-Anderson,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,1,sec,NaN,sec,NaN,NaN,Jarrett Guarantano,no yards,0.0,0.0,1.0,7.0,7.0,NaN,NaN,pass,unranked,unranked
90,401110732,vanderbilt commodores,georgia bulldogs,102876901,vanderbilt commodores,georgia bulldogs,2,2,750,0,14,3,31,62,GREEN,Riley Neal,5.0,Justice Shelton-Mosley,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,1,sec,sec,sec,NaN,3.0,Riley Neal,gain,5.0,0.0,1.0,0.0,14.0,NaN,3.0,pass,unranked,top 5


# check data 

In [68]:
#check for extra downs and drop any above 4

pbp_rank['down'].unique()

for i in range(len(pbp_rank['down'])):
    if pbp_rank['down'].iloc[i]==5:
        print(i)

3988


In [108]:
pbp_rank = pbp_rank.drop(pbp_rank.index[3988])

In [109]:
pbp_rank['run_pass'].unique()

array(['pass', 'run', 'other'], dtype=object)